In [1]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
from IPython.display import Markdown, display, update_display

In [2]:
state = {"next": "", "setup": "", "punchline": "", "rating": ""}

In [3]:
# We are familiar with a standard API request
# This is a public API that returns a random joke
# The API developers have a very deterministic response even if random
# Both sides program imperatively
import requests

get_random_joke_internet = requests.get(
    "https://official-joke-api.appspot.com/random_joke"
)

print(get_random_joke_internet.json())

{'type': 'general', 'setup': 'What’s 50 Cent’s name in Zimbabwe?', 'punchline': '200 Dollars.', 'id': 149}


In [ ]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")
# google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key not set


In [5]:
client = OpenAI()
claude = anthropic.Anthropic()

In [6]:
# original propmt
system_message = """
You are an assistant that is great at telling jokes.
"""

# Here is where we can do some AI programming
ai_programming = """
A joke worthy of publishing is a joke that has a rating of 8.5/10 or above.

If the joke is worthy of publishing also include next: PUBLISH otherwise next: RETRY

Here is an example of a joke worth of publishing:
Supply the respoinse in the following format:
|{"setup": "The setup of the joke",
"punchline": "The punchline of the joke",   
"rating": "9.0",
"next": "PUBLISH"
}|

Remove all back ticks and other unnecessary characters and just print the JSON format and nothing else.

Please ensure jokes are not repeated on retries

Thank you.

"""

system_message += ai_programming

user_prompt = "Tell a light-hearted joke for an audience of Pythonistas in Brighton"

In [7]:
prompts = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt},
]

In [8]:
# GPT-3.5-Turbo

response = client.chat.completions.create(model="gpt-3.5-turbo", messages=prompts)
print(response.choices[0].message.content)
output = response.choices[0].message.content.replace("\n", "")

|{"setup": "Why do Python programmers prefer dark mode?", 
"punchline": "Because light attracts bugs!", 
"rating": "8.7", 
"next": "PUBLISH"
}|


In [9]:
# print(output)
output_json = output.split("|")
# print(output_json[1])
output_dict = json.loads(output_json[1])
print(type(output_dict))
print(output_dict)
if output_dict["next"] == "PUBLISH":
    print("PUBLISH: TRUE")
    state["next"] = output_dict["next"]
    state["setup"] = output_dict["setup"]
    state["rating"] = output_dict["rating"]
    state["punchline"] = output_dict["punchline"]

else:
    print("RETRY")
    state["next"] = "RETRY"

<class 'dict'>
{'setup': 'Why do Python programmers prefer dark mode?', 'punchline': 'Because light attracts bugs!', 'rating': '8.7', 'next': 'PUBLISH'}
PUBLISH: TRUE


In [10]:
print(state)

{'next': 'PUBLISH', 'setup': 'Why do Python programmers prefer dark mode?', 'punchline': 'Because light attracts bugs!', 'rating': '8.7'}
